In [46]:
import numpy as np
import pandas as pd
import csv
import os
import time
import datetime
print(os.listdir("."))

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
np.random.seed(0)

import warnings
warnings.filterwarnings('ignore')


['.config', 'drive', 'sample_data']


In [32]:
import tensorflow as tf
# Go to Edit > Notebook Settings > Hardware accelerator > GPU
device_name = tf.test.gpu_device_name()
print(device_name)

import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('We will use {} {}GPU(s)'.format(torch.cuda.device_count(), torch.cuda.get_device_name() ))
else:
  device = torch.device('cuda')
  print('No GPU(s) available; we will use "cpu"')

device=torch.device('cuda')

/device:GPU:0
We will use 1 Tesla P100-PCIE-16GBGPU(s)


In [33]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
DRIVE_PATH = "/content/drive/My Drive/MIDS/docSimilarity/"

print(os.listdir(DRIVE_PATH))

['CosineSimilarity.ipynb', 'getTwitterData_TweePy.ipynb', 'cbikkanur_twitter_credentials.json', 'cbikkanur_twitter_credentials.ipynb', 'tweets_04232020.csv', 'SearchTweets.csv', 'BankTweets.csv', 'documentSimilarity.ipynb', 'tweets_04242020.csv', 'SearchTweets_04242020.csv', 'BankTweets_04242020.csv', 'SearchTweets_04252020.csv', 'tweets_04252020.csv', 'BankTweets_04252020.csv', 'documentSimilarity_TfIdf.ipynb.txt', 'SearchTweets_04262020.csv', 'BankTweets_04262020.csv', 'tweets_04262020.csv', 'getTwitterData.ipynb', 'documentSimilarity_TfIdf.ipynb', '.ipynb_checkpoints']


In [35]:
all_files = [DRIVE_PATH+'tweets_04232020.csv', DRIVE_PATH+'./tweets_04242020.csv', DRIVE_PATH+'./tweets_04252020.csv', DRIVE_PATH+'./tweets_04262020.csv']

df_list = []
for filename in all_files:
    df_list.append(pd.read_csv(filename, sep=',',header=0, encoding='utf-8', index_col = 0))
    
df_tweets = pd.concat(df_list, ignore_index=True)
df_tweets    

,Tweet_Id,User_Id,User_Name,User_Screen_Name,Theme,Text
0,1253375593271394312,80374332,Citibank,Citibank,Bank/Financial,Registering for online access and activating y...
1,1253322747851296768,80374332,Citibank,Citibank,Bank/Financial,Protect your CARES Act payments: Validate comm...
2,1253066045700681731,80374332,Citibank,Citibank,Bank/Financial,It’s simple to set up a payment account in the...
3,1252960358454767616,80374332,Citibank,Citibank,Bank/Financial,We’re committed to helping provide the support...
4,1252673469852143618,80374332,Citibank,Citibank,Bank/Financial,Protect Yourself from COVID-19 Scams: Don’t re...
...,...,...,...,...,...,...
11995,1254444893642149890,1230136435237363713,TRUMP IS THE COVID19 MURDERER!,sellers_hank,coffee,@secupp @sarah310 Putting that sugary crap in ...
11996,1254444893096808449,32665279,¯\_(ツ)_/¯ Anonymous,tildejac,coffee,@Grady_Booch @ADMGoldfish Just don't knock ove...
11997,1254444890936602625,15063195,Jonathan Middleton,jpmiddleton,coffee,RT @joncoopertweets: I literally spit out my c...
11998,1254444888516632578,77998665,cocotu,cocotucafe,coffee,You have to visit Quillabamba in Peru. Beautif...


In [36]:
df_tweets.drop_duplicates(["Text", "User_Id"], inplace=True) # remove duplicates in place and reset index
df_tweets = df_tweets.reset_index(drop=True)
df_tweets.shape

(6498, 6)

In [37]:
df = pd.DataFrame()
df['text'] = df_tweets['Text']
df.tail()

,text
6493,@secupp @sarah310 Putting that sugary crap in ...
6494,@Grady_Booch @ADMGoldfish Just don't knock ove...
6495,RT @joncoopertweets: I literally spit out my c...
6496,You have to visit Quillabamba in Peru. Beautif...
6497,RT @spacecowbot: let it be known that i do not...


In [38]:
total_indices_size = df.shape[0]
train_size = 0.8
train_indices_size = int(train_size * total_indices_size)
test_indices_size = total_indices_size - train_indices_size
print('total records: {}\n trainig_set record: {}\n test_set records: {}'.format(total_indices_size, train_indices_size, test_indices_size))

total_indices_array = np.array([x for x in range(total_indices_size)])
np.random.shuffle(total_indices_array)
train_indices, test_indices = total_indices_array[:train_indices_size], total_indices_array[train_indices_size:]

total records: 6498
 trainig_set record: 5198
 test_set records: 1300


In [39]:
train_set, test_set = df.loc[train_indices], df.loc[test_indices]
train_set[:5]

,text
5602,@Citi Baloney! Worst of the worst. @Citi your...
3609,RT @MaliikObee: ESPN: “They take him with the ...
3520,RT @Paradise_pups: The dog walks you https://t...
1622,If you applied through Small Business Online B...
2443,"@AIRlunchbox @TIDAL @TMobile Nope not you,I am..."


In [40]:
# Documentation for transformers: https://huggingface.co/transformers/index.html
! pip install transformers

In [41]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states = True ) # return hidden layers
model.cuda

<bound method Module.cuda of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [0]:
def vectorize_sentence(tokenizer, model, text):
  MAX_LENGTH = 140
  input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=MAX_LENGTH) # encode for tokenizing
  padded_sequences = pad_sequences([input_ids], maxlen=MAX_LENGTH, dtype='long', truncating='post', padding='post') # pad 0 at the end of sentence until MAX_LENGTH
  input_ids = padded_sequences[0] # remove the outer list
  attention_mask = [int(i>0) for i in input_ids]

  input_ids = torch.tensor(input_ids)
  attention_mask = torch.tensor(attention_mask)

  input_ids = input_ids.unsqueeze(0) # Extra dimension for batch
  attention_mask = attention_mask.unsqueeze(0)  # Extra dimension for batch

  model.eval() # Only forward pass
  input_ids.to(device) # https://github.com/huggingface/transformers/blob/master/notebooks/02-transformers.ipynb
  attention_mask.to(device)

  with torch.no_grad(): # No gradient descent n
    logits, encoded_layers = model(input_ids = input_ids, token_type_ids = None, attention_mask = attention_mask)
    last_layer = 12
    batch = 0
    last_token_index = 0 

    vector = encoded_layers[last_layer][batch][last_token_index]
    vector = vector.detach().cpu().numpy()

    return vector

In [110]:
sample_text = test_set.iloc[2].text
sample_tokens = tokenizer.tokenize(sample_text)
print('sample text: \n{} \n\n BERT tokens for sample text: \n{}'.format(sample_text, sample_tokens))

sample text: 
@PSCC_Tualatin Hi Jonathan, the applications are held in submission order for processing if additional funding becomes available from Congress. Thank you for your continued patience. -EC 

 BERT tokens for sample text: 
['@', 'ps', '##cc', '_', 'tu', '##ala', '##tin', 'hi', 'jonathan', ',', 'the', 'applications', 'are', 'held', 'in', 'submission', 'order', 'for', 'processing', 'if', 'additional', 'funding', 'becomes', 'available', 'from', 'congress', '.', 'thank', 'you', 'for', 'your', 'continued', 'patience', '.', '-', 'ec']


In [111]:
sample_vector = vectorize_sentence(tokenizer, model, sample_text)
print('Shape of the vector: {}'.format(sample_vector.shape))

Shape of the vector: (768,)


In [47]:
def format_time(elapsed):
  return str(datetime.timedelta(seconds = int(round(elapsed))))


0:06:00


In [0]:
def vectorize_dataset(df):
  start_time = time.time()
  embeddings = []
  num_records = df.shape[0]
  row_num = 0
  for index, row in df.iterrows():
    text = row['text']
    if row_num % 200 == 0 and not row_num == 0:
      elapsed = format_time(time.time() - start_time)
      rows_per_sec = (time.time() - start_time) / row_num
      remaining_sec = rows_per_sec * (num_records - row_num)
      remaining = format_time(remaining_sec)
      print(' comment {:>7,} of {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(row_num, num_records, elapsed, remaining))
    embeddings.append(vectorize_sentence(tokenizer, model, text))
    row_num += 1
  return  embeddings 

In [63]:
train_set_embeddings = vectorize_dataset(train_set)
train_vectors = np.stack(train_set_embeddings)
train_vectors.shape

 comment     200 of   5,198.    Elapsed: 0:01:32. Remaining: 0:38:12
 comment     400 of   5,198.    Elapsed: 0:03:05. Remaining: 0:37:03
 comment     600 of   5,198.    Elapsed: 0:04:37. Remaining: 0:35:25
 comment     800 of   5,198.    Elapsed: 0:06:10. Remaining: 0:33:55
 comment   1,000 of   5,198.    Elapsed: 0:07:47. Remaining: 0:32:38
 comment   1,200 of   5,198.    Elapsed: 0:09:21. Remaining: 0:31:08
 comment   1,400 of   5,198.    Elapsed: 0:10:53. Remaining: 0:29:30
 comment   1,600 of   5,198.    Elapsed: 0:12:23. Remaining: 0:27:50
 comment   1,800 of   5,198.    Elapsed: 0:13:56. Remaining: 0:26:18
 comment   2,000 of   5,198.    Elapsed: 0:15:31. Remaining: 0:24:48
 comment   2,200 of   5,198.    Elapsed: 0:17:04. Remaining: 0:23:15
 comment   2,400 of   5,198.    Elapsed: 0:18:34. Remaining: 0:21:39
 comment   2,600 of   5,198.    Elapsed: 0:20:04. Remaining: 0:20:03
 comment   2,800 of   5,198.    Elapsed: 0:21:35. Remaining: 0:18:29
 comment   3,000 of   5,198.    El

(5198, 768)

In [64]:
test_set_embeddings = vectorize_dataset(test_set)
test_vectors = np.stack(test_set_embeddings)
test_vectors.shape


 comment     200 of   1,300.    Elapsed: 0:01:27. Remaining: 0:08:00
 comment     400 of   1,300.    Elapsed: 0:02:59. Remaining: 0:06:42
 comment     600 of   1,300.    Elapsed: 0:04:27. Remaining: 0:05:12
 comment     800 of   1,300.    Elapsed: 0:05:55. Remaining: 0:03:42
 comment   1,000 of   1,300.    Elapsed: 0:07:21. Remaining: 0:02:12
 comment   1,200 of   1,300.    Elapsed: 0:08:45. Remaining: 0:00:44


(1300, 768)

In [108]:
output_dir = './drive/My Drive/MIDS/docSimilarity/model_save/'
print(os.listdir("."))
if not os.path.exists(output_dir):
  print('directory does not exist')
  os.makedirs(output_dir)

np.save('train_vectors.npy', train_vectors)  
np.save('test_vectors.npy', test_vectors)  

! cp -r ./train_vectors.npy './drive/My Drive/MIDS/docSimilarity/model_save/'
! cp -r ./test_vectors.npy './drive/My Drive/MIDS/docSimilarity/model_save/'

['.config', 'train_vectors.npy', 'drive', 'test_vectors.npy', 'model_save', 'sample_data']


In [112]:
# Sanity check
cosine_similarities = cosine_similarity(sample_vector.reshape(1, -1), train_vectors).flatten()
top_Indices = cosine_similarities.argsort()[::-1][:1].item(0)
top_tweet_Indices = train_indices[top_Indices]

print("\nInput Text:\n", test_set.iloc[2].text, "\n")
print("\nSimilar Text:\n", df_tweets.loc[top_tweet_Indices, "Text"], "\n")


Input Text:
 @PSCC_Tualatin Hi Jonathan, the applications are held in submission order for processing if additional funding becomes available from Congress. Thank you for your continued patience. -EC 


Similar Text:
 @Addvalue2others Hi, the applications are held in submission order for processing if additional funding becomes available from Congress. Thank you for your continued patience. -EC 



In [0]:
def find_similar(train_vectors, train_indices, test_vectors, test_indices, test_set_df, df_tweets):
    similar_text_list = []
    similar_score_list = []
    similar_theme_list = []
    for vector in test_vectors:
        cosine_similarities = cosine_similarity(vector.reshape(1, -1), train_vectors).flatten()
        similar_doc_index =  cosine_similarities.argsort()[::-1][:1].item(0)
        top_tweet_Index = train_indices[similar_doc_index]
        similar_text_list.append(df_tweets.loc[top_tweet_Index, "Text"])
        similar_score_list.append(cosine_similarities[similar_doc_index])
        similar_theme_list.append(df_tweets.loc[top_tweet_Index, "Theme"])    
    
    df = test_set_df 
    df["original_theme"] = df_tweets.loc[test_indices, "Theme"]
    df["similar_text"] = similar_text_list
    df["similar_score"] = similar_score_list     
    df["similar_theme"] = similar_theme_list
    return df

In [93]:
similar_test_df = find_similar(train_vectors, train_indices, test_vectors, test_indices, test_set, df_tweets)
similar_test_df.shape

(1300, 5)

In [95]:
similar_test_df

,text,original_theme,similar_text,similar_score,similar_theme
2725,RT @MikeDentale: Airlines sending me “we’re in...,airlines,RT @MikeDentale: Airlines sending me “we’re in...,1.000000,airlines
2454,@Yo_ItsMONROE EVERY! One. The one where i live...,car,RT @freaknastywet: Y’all like car play? 😈\nhtt...,0.956251,car
1459,"@PSCC_Tualatin Hi Jonathan, the applications a...",Bank/Financial,"@Addvalue2others Hi, the applications are held...",0.993342,Bank/Financial
3867,RT @AsemWORLD: Freestyle Friday S2Ep11 \nClima...,tokyo,RT @AsemWORLD: Freestyle Friday S2Ep11 \nClima...,1.000000,tokyo
1903,#Citi appoints Kristine Braden as Head of the ...,Bank/Financial,#Citi is proud to be named Best Global Bank fo...,0.960067,Bank/Financial
...,...,...,...,...,...
4931,A team at NASA developed a ventilator tailored...,nasa,"Coronavirus live updates: U.S. passes 50,000 d...",0.944459,covid-19
3264,RT @Citi: Jane Fraser discusses how business l...,@Citi,RT @Citi: Jane Fraser discusses how business l...,1.000000,@Citi
1653,Clients and small business owners: if you've b...,Bank/Financial,Protect Yourself from COVID-19 Scams: Be wary ...,0.954953,Bank/Financial
2607,isaiah got him and i matching tokyo ghoul shir...,tokyo,So #TheSelection is 8 years old today. Seeing ...,0.897029,cake


In [94]:
accuracy = 100 * np.sum(similar_test_df["original_theme"] == similar_test_df["similar_theme"])/similar_test_df.shape[0]
print("Accuracy on test data in predicting theme: {}%".format(np.round(accuracy,2)))

Accuracy on test data in predicting theme: 72.77%


In [0]:
similar_test_df.to_csv("similar_test_tweets.csv")

In [0]:
os.listdir('.')
! cp -r ./similar_test_tweets.csv './drive/My Drive/MIDS/docSimilarity/model_save/'